In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [157]:
### Problem, this method does not exclude questions. ###
def call_pull(ticker, year, quarter,key = 'e46f1a303dafb62460de104424a00084'):
    transcript = requests.get(f'https://financialmodelingprep.com/api/v3/earning_call_transcript/{ticker}?quarter={quarter}&year={year}&apikey={demo}').json()
    transcript = transcript[0]['content']
    return transcript

def entire_report_pull(stock_list, year_list, quarter_list):
    report = []
    columns = []
    for stock in stock_list:
        stonks = []
        for yr in year_list:
            for qtr in quarter_list:
                if f'{yr},{qtr} Qtr' not in columns:
                    columns.append(f'{yr},{qtr} Qtr')
                try:
                    stonks.append(call_pull(stock, yr, qtr))
                except IndexError:
                    stonks.append(None)
        report.append(stonks)
    return pd.DataFrame(report, columns = columns, index = stock_list)


test_tech = ['AAPL','AMZN']
test_quarters= [1,2,3,4]
test_years = [2019,2020]


In [174]:
def better_entire_report_pull(stock_list, year_list, quarter_list):
    report = []
    for stock in stock_list:
        for yr in year_list:
            for qtr in quarter_list:
                stonks = []
                stonks.append(stock)
                stonks.append(f'{yr},{qtr} Qtr')
                try:
                    stonks.append(call_pull(stock, yr, qtr))
                except:
                    stonks.append(None)
                report.append(stonks)
    return pd.DataFrame(report, columns = ['Ticker', 'Date', 'Transcript'])

In [49]:
better_entire_report_pull(test_tech, test_years, test_quarters)

,Ticker,Date,Transcript
0,AAPL,"2019,1 Qtr",Operator: Good day and welcome to the Apple In...
1,AAPL,"2019,2 Qtr","Operator: Good day, and welcome to the Apple I..."
2,AAPL,"2019,3 Qtr","Operator: Good day, and welcome to the Apple I..."
3,AAPL,"2019,4 Qtr","Operator: Good day, everyone. Welcome to the A..."
4,AAPL,"2020,1 Qtr","Operator: Good day, everyone. Welcome to the A..."
5,AAPL,"2020,2 Qtr",Operator: Good day everyone. Welcome to the Ap...
6,AAPL,"2020,3 Qtr","Operator: Good day, everyone. Welcome to the A..."
7,AAPL,"2020,4 Qtr",Operator: Good day everyone and welcome to th...
8,AMZN,"2019,1 Qtr","Operator: Thank you for standing by. Good day,..."
9,AMZN,"2019,2 Qtr","Operator: Thank you for standing by. Good day,..."


In [13]:
test_tech = ['AAPL','AMZN']
test_quarters= [1,2,3,4]
test_years = [2019,2020]

def label_pull(ticker, limit=13,key = 'e46f1a303dafb62460de104424a00084'):
    label = requests.get(f'https://financialmodelingprep.com/api/v3/enterprise-values/{ticker}?period=quarter&limit={limit}&apikey=e46f1a303dafb62460de104424a00084').json()
    return pd.DataFrame(label)

def better_than_VGT(row):
    ### to be used in next function ###
    if row['Percent Change'] >= row['VGT']:
        return 1
    else:
        return 0

def full_pull(stock_list, limit=13):
    
    ### form a DataFrame that shows quarterly stock movement ###
    label = pd.DataFrame()
    skipped = []
    for stock in stock_list:
        try: 
            label = label.append(label_pull(stock, limit).iloc[::-1])
        
        except:
            skipped.append(stock)
            continue
            
    new_label = label.iloc[:,:3]
    
    ### add Percent Chane ###
#     new_label['Percent Change'] = new_label.stockPrice.pct_change()
#     new_label['Percent Change'][(new_label.date == '2017-12-30')|(new_label.date == '2017-12-31')] = np.nan

    
    ### Retrieved from https://institutional.vanguard.com/web/c1/investments/product-details/fund/0958 ###
    ### Returns from 2017-2020 all quarters ###
#     VGT = pd.DataFrame()
#     for stock in stock_list:
#         VGT = VGT.append(pd.DataFrame([.0855, .0411, .0637, .1194, -.1738, .2068, .0543, .0261, .1393, -.1315, .3185, .1202, .1389], columns = ['VGT']))
#     VGT.reset_index(inplace=True)
#     new_label.reset_index(inplace=True)
    
#     ### drop NaN values ###
#     combined_df = pd.concat([new_label, VGT], axis=1).drop('index', axis=1)
#     combined_df.dropna(inplace=True)
    
#     ### Create final column to be used as label ###
#     combined_df['Better than VGT?'] = (combined_df['Percent Change'] > combined_df['VGT']).astype(int)
    
    return new_label, skipped

y = full_pull(['AAPL'])
y

(   symbol        date  stockPrice
 12   AAPL  2017-12-30   42.307499
 11   AAPL  2018-03-31   41.669998
 10   AAPL  2018-06-30   46.277500
 9    AAPL  2018-09-29   56.435001
 8    AAPL  2018-12-29   39.057499
 7    AAPL  2019-03-30   47.487499
 6    AAPL  2019-06-29   49.480000
 5    AAPL  2019-09-28   54.705002
 4    AAPL  2019-12-28   72.449997
 3    AAPL  2020-03-28   61.935001
 2    AAPL  2020-06-27   88.407501
 1    AAPL  2020-09-26  112.279999
 0    AAPL  2020-12-26  136.690002,
 [])

In [16]:
tech = ['ACIW','ACLS','ACMR','AGYS','ALGM',
'ALRM','ALTR','AMBA','AMKR','AMSWA','APPF','APPN','APXT',
'ATOM','AVID','AVNW','AVT','AXTI','AZPN','BAND','BBSI',
'BCOV','BIGC','BL','BLKB','BMBL','BNFT','BRKS','BTRS',
'CAMP','CARG','CASA','CCMP','CCRN','CDLX','CERT','CEVA',
'CHNG','CHX','CLSK','CMTL','COMM','CPSI','CRNC','CRSR',
'CRUS','CSGS','CSOD','CVLT','DCBO','DCT','DGII','DIOD',
'DMRC','DOMO','DSGX','DSP','DSPG','EBIX','EPAY','ERII',
'EVBG','EVER','EXTR','FEYE','FORM','FROG','FSLR','FSRV',
'GDYN','GFN','GLUU','GNOG','GRPN','GTYH','HCAT','HLIT',
'HSII','HSTM','ICHR','INOV','INTZ','IPHI','JAMF','JCOM',
'KE','KELYA','KELYB','KFRC','KOPN','LASR','LCY','LECO',
'LORL','LPSN','LSCC','LTRPA','LTRPB','MANH','MARA','MCFE',
'MDRX','MGNI','MGRC','MIDD','MITK','MSGM','MSTR','MTSI',
'NATI','NCNO','NH','NTCT','NTNX','NVEC','NXGN','OMCL',
'OSIS','OSPN','PCTY','PDFS','PEGA','PFPT','PLAB','PLUS',
'PLXS','POWI','PRCH','PRFT','PRGS','PS','PUBM','QADA',
'QADB','QLYS','QMCO','RBBN','RCII','RIOT','RMBS','RP',
'RPD','RXT','SABR','SANM','SCPL','SCSC','SCWX','SGH',
'SGMS','SHLS','SITM','SLAB','SLP','SMCI','SMTC','SPSC',
'SPT','SPWR','SSPK','SSTI','SSYS','SUMO','SVMK','SWIR',
'SYKE','SYNA','TCX','TENB','TLS','TRIP','TRUE','TTEC',
'TTMI','TWOU','UCTT','UPLD','UPWK','VECO','VERI','VERX',
'VIAV','VIH','VOXX','VRNS','VRNT','VSAT','VUZI','XPER',
'ZIXI']

stonk = pd.DataFrame()
skipped = []
for stock in tech:
    try:
        stonk = stonk.append(label_pull(stock).iloc[::-1])
    except:
        skipped.append(stock)
        continue
        
stonk

,symbol,date,stockPrice,numberOfShares,marketCapitalization,minusCashAndCashEquivalents,addTotalDebt,enterpriseValue
12,ACIW,2017-12-31,22.69,118315000.0,2.684567e+09,69710000.0,685729000.0,3.300586e+09
11,ACIW,2018-03-31,23.21,115642000.0,2.684051e+09,74281000.0,679240000.0,3.289010e+09
10,ACIW,2018-06-30,24.67,115548000.0,2.850569e+09,59033000.0,677799000.0,3.469335e+09
9,ACIW,2018-09-30,27.26,115889000.0,3.159134e+09,76342000.0,674924000.0,3.757716e+09
8,ACIW,2018-12-31,27.67,116066000.0,3.211546e+09,148502000.0,671756000.0,3.734800e+09
...,...,...,...,...,...,...,...,...
4,VUZI,2019-12-31,2.01,30916081.0,6.214132e+07,10606091.0,0.0,5.153523e+07
3,VUZI,2020-03-31,1.23,29786800.0,3.663776e+07,6142658.0,0.0,3.049511e+07
2,VUZI,2020-06-30,2.39,36341616.0,8.685646e+07,13229182.0,1555900.0,7.518318e+07
1,VUZI,2020-09-30,4.56,39837321.0,1.816582e+08,24938858.0,1555900.0,1.582752e+08


In [25]:
new_stonk = stonk.iloc[:,:3]
new_stonk['Percent Change'] = new_stonk.stockPrice.pct_change()
new_stonk['Percent Change'][(new_stonk.date == '2017-12-30')|(new_stonk.date == '2017-12-31')] = np.nan
new_stonk.symbol.value_counts()

/home/daniel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


AMSWA    13
AVNW     13
MSTR     13
VSAT     13
SABR     13
         ..
TLS       1
MSGM      1
DCBO      1
PUBM      1
SHLS      1
Name: symbol, Length: 157, dtype: int64

In [57]:
booly = (new_stonk.symbol.value_counts() == 13)
Truly = booly.index[booly==True]

In [66]:
newer_stonk = new_stonk[new_stonk.symbol.isin(Truly)]
newer_stonk.reset_index(inplace=True)
newer_stonk.symbol.unique()

array(['ACIW', 'ACLS', 'ACMR', 'AGYS', 'ALRM', 'ALTR', 'AMBA', 'AMKR',
       'AMSWA', 'AVID', 'AVNW', 'AVT', 'AXTI', 'AZPN', 'BAND', 'BBSI',
       'BCOV', 'BL', 'BNFT', 'BRKS', 'CAMP', 'CARG', 'CASA', 'CCMP',
       'CCRN', 'CEVA', 'CHX', 'CLSK', 'CMTL', 'COMM', 'CPSI', 'CRUS',
       'CSGS', 'CSOD', 'CVLT', 'DCT', 'DGII', 'DIOD', 'DMRC', 'DSGX',
       'DSPG', 'EBIX', 'EPAY', 'ERII', 'EVBG', 'EXTR', 'FEYE', 'FORM',
       'GFN', 'GLUU', 'GRPN', 'GTYH', 'HLIT', 'HSII', 'HSTM', 'INTZ',
       'IPHI', 'JCOM', 'KE', 'KELYA', 'KELYB', 'KFRC', 'KOPN', 'LPSN',
       'LSCC', 'LTRPA', 'LTRPB', 'MANH', 'MARA', 'MDRX', 'MGRC', 'MITK',
       'MSTR', 'MTSI', 'NATI', 'NH', 'NTCT', 'NTNX', 'NVEC', 'OMCL',
       'OSIS', 'OSPN', 'PCTY', 'PDFS', 'PEGA', 'PFPT', 'PLAB', 'PLUS',
       'PLXS', 'PRGS', 'QADA', 'QADB', 'QLYS', 'QMCO', 'RBBN', 'RCII',
       'RIOT', 'SABR', 'SANM', 'SCSC', 'SCWX', 'SGH', 'SGMS', 'SLAB',
       'SLP', 'SMCI', 'SMTC', 'SPSC', 'SPWR', 'SSTI', 'SSYS', 'SYNA',
       'TCX',

In [64]:
VGT = pd.DataFrame()
for stock in newer_stonk.symbol.unique():
    VGT = VGT.append(pd.DataFrame([.0855, .0411, .0637, .1194, -.1738, .2068, .0543, .0261, .1393, -.1315, .3185, .1202, .1389], columns = ['VGT']))
VGT.reset_index(inplace=True)
VGT

,index,VGT
0,0,0.0855
1,1,0.0411
2,2,0.0637
3,3,0.1194
4,4,-0.1738
...,...,...
1646,8,0.1393
1647,9,-0.1315
1648,10,0.3185
1649,11,0.1202


In [93]:
### combine and drop NaN values ###
combined_df = pd.concat([newer_stonk, VGT], axis=1).drop('index', axis=1)
combined_df.dropna(inplace=True)
    
### Create final column to be used as label ###
combined_df['Better than VGT?'] = (combined_df['Percent Change'] > combined_df['VGT']).astype(int)
combined_df

,symbol,date,stockPrice,Percent Change,VGT,Better than VGT?
1,ACIW,2018-03-31,23.21,0.022918,0.0411,0
2,ACIW,2018-06-30,24.67,0.062904,0.0637,0
3,ACIW,2018-09-30,27.26,0.104986,0.1194,0
4,ACIW,2018-12-31,27.67,0.015040,-0.1738,1
5,ACIW,2019-03-31,33.21,0.200217,0.2068,0
...,...,...,...,...,...,...
1646,VUZI,2019-12-31,2.01,-0.118421,0.1393,0
1647,VUZI,2020-03-31,1.23,-0.388060,-0.1315,0
1648,VUZI,2020-06-30,2.39,0.943089,0.3185,1
1649,VUZI,2020-09-30,4.56,0.907950,0.1202,1


In [94]:
New_tech = ['ACIW', 'ACLS', 'ACMR', 'AGYS', 'ALRM', 'ALTR', 'AMBA', 'AMKR',
       'AMSWA', 'AVID', 'AVNW', 'AVT', 'AXTI', 'AZPN', 'BAND', 'BBSI',
       'BCOV', 'BL', 'BNFT', 'BRKS', 'CAMP', 'CARG', 'CASA', 'CCMP',
       'CCRN', 'CEVA', 'CHX', 'CLSK', 'CMTL', 'COMM', 'CPSI', 'CRUS',
       'CSGS', 'CSOD', 'CVLT', 'DCT', 'DGII', 'DIOD', 'DMRC', 'DSGX',
       'DSPG', 'EBIX', 'EPAY', 'ERII', 'EVBG', 'EXTR', 'FEYE', 'FORM',
       'GFN', 'GLUU', 'GRPN', 'GTYH', 'HLIT', 'HSII', 'HSTM', 'INTZ',
       'IPHI', 'JCOM', 'KE', 'KELYA', 'KELYB', 'KFRC', 'KOPN', 'LPSN',
       'LSCC', 'LTRPA', 'LTRPB', 'MANH', 'MARA', 'MDRX', 'MGRC', 'MITK',
       'MSTR', 'MTSI', 'NATI', 'NH', 'NTCT', 'NTNX', 'NVEC', 'OMCL',
       'OSIS', 'OSPN', 'PCTY', 'PDFS', 'PEGA', 'PFPT', 'PLAB', 'PLUS',
       'PLXS', 'PRGS', 'QADA', 'QADB', 'QLYS', 'QMCO', 'RBBN', 'RCII',
       'RIOT', 'SABR', 'SANM', 'SCSC', 'SCWX', 'SGH', 'SGMS', 'SLAB',
       'SLP', 'SMCI', 'SMTC', 'SPSC', 'SPWR', 'SSTI', 'SSYS', 'SYNA',
       'TCX', 'TRIP', 'TRUE', 'TTEC', 'TTMI', 'TWOU', 'UCTT', 'VECO',
       'VERI', 'VIAV', 'VOXX', 'VRNS', 'VRNT', 'VSAT', 'VUZI']

In [18]:
stonk.to_csv('../data/y.csv')

In [182]:
# X = better_entire_report_pull(tech, [2018, 2019, 2020], [1,2,3,4])
# X.to_csv('../data/X.csv')

In [183]:
X.head(50)

,Ticker,Date,Transcript
0,ABST,"2018,1 Qtr",None
1,ABST,"2018,2 Qtr",None
2,ABST,"2018,3 Qtr",None
3,ABST,"2018,4 Qtr",None
4,ABST,"2019,1 Qtr",None
5,ABST,"2019,2 Qtr",None
6,ABST,"2019,3 Qtr",None
7,ABST,"2019,4 Qtr",None
8,ABST,"2020,1 Qtr",None
9,ABST,"2020,2 Qtr",None
